## Introduction: the acoustic  particulate cylinder

The propagation of waves in free space is governed by the 2D Helmholtz equation. We denote by $\mathrm V_n$ and $\mathrm U_n$ the cylindrical Bessel functions

$$
\tag{1}
 \mathrm V_n(k\mathbf r):=\mathrm J_n(kr)\mathrm e^{\mathrm in\theta} 
\quad\text{and}\quad
\mathrm U_n(k \mathbf r):= \mathrm H_n(k r)\mathrm e^{\mathrm in\theta}
$$

where $k$ is the background wavenumber and  $(r,\theta)$ are the polar coordinates of $\mathbf r$, ie $\mathbf r = (r\cos \theta,r\sin\theta)$, $\mathrm J_n$ and $\mathrm H_n$ are respectively the Bessel function of order zero and the Hankel function. 

We consider the scattering from a set of $J$ cylinders, referred to as particles, confined in a circular area of radius $R$. The  incident field $u_i$ can be decomposed in modes:

$$
\tag{2}
u_i(\mathbf r) =  \sum_{n=-\infty}^{+\infty}g_n\mathrm V_n(k\mathbf r),\quad g_n\in\mathbb{C}.
$$

Then the averaged scattered field $\langle u_s\rangle$ (over all possible configurations of the $J$ particles confined in the disc of radius $R$) can be written

$$
\tag{3}
\langle u_s\rangle(\mathbf r) =  \sum_{n=-\infty}^{+\infty} \langle \mathfrak F_n \rangle\mathrm H_n(k\mathbf r), \quad  \langle \mathfrak F_n \rangle = \mathrm T_ng_n
$$

where $\mathrm T_n\in\mathbb{C}$ is related to the effective T-matrix $\mathrm T_{n,m}$ of the effective cylinder of radius $R$. More precisely,

$$
\tag{4}
\mathrm T_{n,m} := \delta_{n,m} \mathrm T_n.
$$

With this definition, the scattering from the effective cylinder of radius $R$ can be summerized in matrix form 

$$
\tag{5}
 \langle \mathbf{F} \rangle = \mathbf T \mathbf g
$$

where we defined the vectors $\langle \mathbf{F} \rangle=( \langle \mathfrak F_n \rangle)_n$ and $\mathbf g = (g_n)_n$.

## Example 2: Monte Carlo validation of the coefficients $\mathrm T_n$ 


In the previous example, we showed how to use the function *t_matrix* to compute the effective T-matrix of the cylinder. The latter relied on the Effective Waves Method which is fast and do not need to compute any average. This package also implements Monte Carlo simulations which allows to validate these results.  

Standard Monte Carlo simulations consists in solving the scattering problem several times, compute the empirical average of the scattered field and then compare the results with the predicted effective properties of the homogenized material. In this section is shown how to run the Monte Carlo simulations. In the next section will be shown in details how the effective T-matrix is sampled and how we make use of the spherical geometry to accelerate the Monte Carlo simulations.

#### Fixed set of parameters

First define the host medium 

In [ ]:
using EffectiveTMatrix

## define host_medium
dimension=2;
host_medium = Acoustic(dimension; ρ=1.0, c=1.0);

Then define properties of the effective cylinder:

In [ ]:
## particulate microstructure of the effective cylinder
particle = Particle(Acoustic(2; ρ=1e-2, c=1.0),Circle(1.0));   # sound soft particle of radius 1.0 
sp_MC = Specie(particle; volume_fraction = .05) 

## radius of the effective cylinder
cylinder_radius = 20.0;

Finally we set the frequency and some parameters required for the Monte Carlo simulations:

In [ ]:
ω=.1                   # frequency
basis_field_order = 4; # sample Tₙ for n ∈ [0;basis_field_order] 

## other Monte Carlo parameters
kws_MC = Dict(
    :basis_order => 5,                       # truncation of the Foldy-Lax equation
    :nb_iterations_max => 6000,              # max number of iterations
    :nb_iterations_step => 100,              # convergence criteria is checked at every steps
    :prec => 5e-2                            # precision of the Monte Carlo simulation
);

The result is stored in an object of type MonteCarloResult that has to be initialise. The function run_MC_validation! runs the Monte Carlo simulation. The result can simply be plot using the function plot:

In [ ]:
MC = MonteCarloResult(ω,sp_MC,cylinder_radius; basis_field_order=basis_field_order); # initialisation
sample_effective_t_matrix!(MC,host_medium; kws_MC...); # Monte Carlo simulations
run_MC_validation!(host_medium, MC; kws_MC...);     # Effective Waves Method
plot(MC)

<p align="center">
  <img src="MC_fixed_params.png" width="500" />
</p>

The plot above shows $\mathrm{Re}(\mathrm T_n)$ for $n\in[0,4]$ using three different methods: the Monte Carlo simulations (MC), the Effective Waves Method (EWM) and the Monopole Approximation of the Effective Waves Method (EFM-MA). The latter method is obtained when the particles are approximated with monopole scatterers and give good results in the case of quasi-monopole scatterers (cf. next paragraph).

#### List of parameters

Running Monte Carlo simulations on a list of parameters is very similar to the above, one only need to initialise a vector of MonteCarloResult objects instead. Here is an example of a multi frequencies simulation:

In [ ]:
## MC loop on frequency
Ω = collect(.1:.05:1.5);
MC_vec = [MonteCarloResult(ω,sp_MC,cylinder_radius) for ω ∈ Ω];
sample_effective_t_matrix!(MC_vec,host_medium; kws_MC...); # Monte Carlo simulations
run_MC_validation!(host_medium, MC_vec; kws_MC...);     # Effective Waves Method

# plot results
plot(MC_vec)

<p align="center">
  <img src="MC_multi_frq.png" width="500" />
</p>

The Monopole approximation method (EWM-MA) gives a good approximation in the low frequency regime because the sound soft particles are then quasi-monopole scatterers.